In [2]:
!pip install sagemaker ipywidgets --upgrade --quiet

## Dataset preparation

In [3]:
import numpy as np

# Load the dataset
with np.load("dataset/100P100A.npz") as data:
    train_faces = data["arr_0"]
    train_labels = data["arr_1"]
    

## Upload the database.npz to s3

In [8]:
import numpy as np
import pandas as pd
import boto3

data = np.load('dataset/100P100A.npz')  # Replace 'your_file.npz' with the actual file path

bucket_name = 'actors-dataset'
folder_name = 'train'
file_name = 'database.npz'  # The name you want for the file in the bucket

# Create an S3 client
s3 = boto3.client('s3')

# Upload the loaded data to S3
s3.upload_file('dataset/100P100A.npz', bucket_name, f'{folder_name}/database.npz')

print(f"{file_name} uploaded to {bucket_name}")


database.npz uploaded to actors-dataset


In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

sagemaker_session=sagemaker.Session()
role = sagemaker.get_execution_role()

train_data_location = 's3://{}/train'.format(bucket_name)
output_path = 's3://{}/output'.format(bucket_name)

estimator = TensorFlow(entry_point='training_script.py',
                       role=role,
                       train_instance_count=1,
                       train_instance_type='ml.p2.xlarge',
                       framework_version='2.6.0',
                       py_version='py38',
                       output_path=output_path,
                       hyperparameters={"epochs": 10},
                       sagemaker_session=sagemaker_session)

estimator.fit({'train': train_data_location})

In [ ]:
print(output_path)

## Deploy the model

In [ ]:
print(estimator.model_data)

In [ ]:
from sagemaker.tensorflow import TensorFlow, TensorFlowModel

tensorflow_model = TensorFlowModel(
    model_data=estimator.model_data,
    role=role,
    framework_version="2.6.0",
    sagemaker_session=sagemaker_session
)

In [ ]:
predictor = tensorflow_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name="actor-recognizer-endpoint" 
)

## Inference Example
This is an inference example using the deployed model through SageMaker.

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist



# Perform Inference using the deployed model

result = predictor.predict(sample)

print(result['predictions'])

predicted_label = np.argmax(result['predictions'])
print("Predicted Label:", predicted_label)

actual_label = y_test[sample_index]
print("Actual Label:", actual_label)
